# Interpretation of fine-tuned DistilProtBERT using [**Captum**](https://captum.ai/)

Source code info:

Used notebook: https://gist.github.com/davidefiocco/3e1a0ed030792230a33c726c61f6b3a5 (adaptation to classification from original tutorial on question answering: https://captum.ai/tutorials/Bert_SQUAD_Interpret)

(Used notebook is based on this github issue: https://github.com/pytorch/captum/issues/303)

Related github issue: https://github.com/pytorch/captum/issues/249

## Load initial libraries, models, data:

In [1]:
!pip install transformers datasets tokenizers evaluate captum --quiet

In [2]:
TOKENIZER = 'EvaKlimentova/knots_distillprotbert_alphafold'
HF_MODEL_NAME = 'EvaKlimentova/knots_distillprotbert_alphafold'
INPUT_CSV = '/home/jovyan/data/proteins_m1/minimums_p40_for_interpretation.csv'

In [3]:
import numpy as np
import torch

torch.no_grad()

In [4]:
import tensorflow as tf
from transformers import BertTokenizer, BertForSequenceClassification
from captum.attr import visualization as viz
from captum.attr import LayerIntegratedGradients

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')  # solves all CUDA out of memory problems :)
device

2023-02-05 14:58:41.865804: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-05 14:58:43.625878: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-05 14:58:43.625961: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-05 14:58:43.625969: W tensorfl

device(type='cpu')

### Load the tokenizer:

In [5]:
tokenizer = BertTokenizer.from_pretrained(TOKENIZER, max_length=1024, truncation=True, num_labels=2)
tokenizer

PreTrainedTokenizer(name_or_path='EvaKlimentova/knots_distillprotbert_alphafold', vocab_size=30, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

### Load the model:

In [6]:
model = BertForSequenceClassification.from_pretrained(HF_MODEL_NAME, output_attentions=True)
model.to(device)
model.eval()
model.zero_grad()
# model

### Load the data:

In [7]:
import csv
import pandas as pd

with open(INPUT_CSV, newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

df = pd.DataFrame(data[1:], columns=data[0])
#df = df.set_index('id')
#df.index = df.index.astype('str')  # df.loc[df['id'] == 'A0A2G6G956']   (index: 3853)
df

,id,sequence_str,sequence_len,sequence_pred,family,knot_start,knot_end,knot,knot_str,patch,min_sequence_str,min_pred,min_start,min_end,overlap_pred,drop_difference
0,A0A0X3YDV6,MVSKLGWQLVQLGRRLWVRTVAFAVLALFSALVAVLVQDYIPETLS...,416,1.0,DUF,184,335,PWLQAQPLQSLQQIPKDAKPVLTNTIGYLQLIDIKAINKWAAENNC...,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,AEIASKALSPAINDPGTAIEIIGRGMRVLSNWQAPSMDKA,MVSKLGWQLVQLGRRLWVRTVAFAVLALFSALVAVLVQDYIPETLS...,0.99658203125,285,325,1.0,0.00341796875
1,J3D4J7,MANDFLFTSESVSEGHPDKVADQISDAILDAIFKQDPRSRVAAETL...,303,0.9995117,AdoMet synthase,15,279,HPDKVADQISDAILDAIFKQDPRSRVAAETLTNTGLVVLAGEITTN...,XXXXXXXXXXXXXXXHPDKVADQISDAILDAIFKQDPRSRVAAETL...,VDTYGGACPHGGGAFSGKDPSKVDRSAAYAARYVAKNIVA,MANDFLFTSESVSEGHPDKVADQISDAILDAIFKQDPRSRVAAETL...,0.0007562637329101,257,297,0.55,0.99875543626709
2,A0A645GPG4,MEDKYLKRAGLDYWSLVEIKYHDNLDAFFDMYREGKFFLSTTKAKN...,115,1.0,SPOUT,38,80,LSTTKAKNKYTDLKYEKDCFILFGKETAGLPKDLLLKNPDEC,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXLSTTKAKN...,FILFGKETAGLPKDLLLKNPDECIRVPMIDEARSLNLSNS,MEDKYLKRAGLDYWSLVEIKYHDNLDAFFDMYREGKFFLSTTKAKN...,0.0019111633300781,57,97,0.575,0.9980888366699219
3,A0A2G5ETQ7,MCFNGKRQSPIEIVKKNTVFDQNLGPLIVGYNDASATLINNGFNVE...,212,0.99853516,Carbonic anhydrase,8,207,SPIEIVKKNTVFDQNLGPLIVGYNDASATLINNGFNVELRYENDVG...,XXXXXXXXSPIEIVKKNTVFDQNLGPLIVGYNDASATLINNGFNVE...,PIEIVKKNTVFDQNLGPLIVGYNDASATLINNGFNVELRY,MCFNGKRQSXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,0.0004529953002929,9,49,1.0,0.9980821646997071
4,A0A3G9G4U1,MRRPVLLFTVAAFALMSAGLSSCGKPKAEHGDPHAEAAGEHGGGDH...,284,0.99902344,Carbonic anhydrase,89,280,PINLTGVAAPKSVNLTLDYTSSPAKIQNLGHAIQVSPTDGGGVVMD...,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,TWLVYASPLSISPEQVDAYQRLTGPNARPIQPPQGRDILH,MRRPVLLFTVAAFALMSAGLSSCGKPKAEHGDPHAEAAGEHGGGDH...,0.046966552734375,240,280,1.0,0.952056887265625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6589,A0A3S5HM02,MPKEIAAPEFKPYIPASAVLPEFTLRALVMGVVLGMIFGASSLYLV...,757,0.99902344,membrane,97,693,FGLGVTMPAILILGFDLEISRVALVAVLGGLLGILLMIPMRRAMIV...,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,TAGSAGESIAFGLGVTMPAILILGFDLEISRVALVAVLGG,MPKEIAAPEFKPYIPASAVLPEFTLRALVMGVVLGMIFGASSLYLV...,0.998046875,87,127,0.75,0.0009765649999999848
6590,A0A3B4X9R3,MSPDCWKKRQNPQIKRFPMETGLVNVFRPVQELNERQVSASPATPL...,355,0.99902344,Carbonic anhydrase,80,325,PINIVTKTAVIDEHLDAFTYTKFDDKNTIKSITNSGHSVKCVLKED...,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,SSEVSVTEEISIDDLLGNVNRAAYYRYNGSLTTPSCNEAV,MSPDCWKKRQNPQIKRFPMETGLVNVFRPVQELNERQVSASPATPL...,0.958984375,244,284,1.0,0.040039064999999985
6591,A0A1X0QVI8,MASQENVTKDELQYSEKKEEEIINEKETKGQEEFWLKDTSDVKAVE...,761,0.9995117,membrane,161,716,SAYGTSVLSTQQLYFNRTPGVAGSIFFLFSTQLIGYGIAGQLRSFM...,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,KHDDEHYFNADETDLAIVNEIAITEDTPNVPIITVRAIVV,MASQENVTKDELQYSEKKEEEIINEKETKGQEEFWLKDTSDVKAVE...,0.9990234375,49,89,0.0,0.00048826250000000293
6592,A0A6M2AUN7,MAYLFTSESVSEGHPDKVADQISDALLDNFLAFDPESKVACETLVT...,418,0.9995117,AdoMet synthase,417,417,,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX...,MAYLFTSESVSEGHPDKVADQISDALLDNFLAFDPESKVA,XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXCETLVT...,0.0023307800292968,0,40,0.0,0.9971809199707032


In [8]:
def tokenize_function(s):
    seq_split = ' '.join(s)
    return tokenizer(seq_split, return_tensors='pt')

## Captum interpretation:

In [40]:
# def predict(inputs):
#     return model(inputs).logits

# def custom_forward(inputs):
#     preds = predict(inputs)
#     return torch.softmax(preds, dim = 1)[0][0].unsqueeze(-1)

def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    return model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask, ).logits

def custom_forward(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    preds = predict(inputs,
                    token_type_ids=token_type_ids,
                    position_ids=position_ids,
                    attention_mask=attention_mask)
    return torch.softmax(preds, dim = 1)[0][0].unsqueeze(-1)

Compute attributions with respect to the `BertEmbeddings` layer:

1. define baselines/ references,
2. numericalize baselines and inputs.

In [10]:
REF_TOKEN_ID = tokenizer.pad_token_id  # token used for generating token reference
SEP_TOKEN_ID = tokenizer.sep_token_id  # token added to the end of the input text
CLS_TOKEN_ID = tokenizer.cls_token_id  # token used at the beginning of the input text
print(f'REF={REF_TOKEN_ID}, SEP={SEP_TOKEN_ID}, CLS={CLS_TOKEN_ID}')

REF=0, SEP=3, CLS=2


Helper function for getting one sequence in raw, tokenized, patched, and patched_tokenized formats:

In [11]:
def get_sample(dataset, index):
    seq_info = df.iloc[index]
    seq_id = seq_info['id']
    seq_len = int(seq_info['sequence_len'])
    
    raw_seq = seq_info['sequence_str']
    tokenized_seq = tokenize_function(raw_seq)
    
    patched_seq = seq_info['min_sequence_str']
    tokenized_patched_seq = tokenize_function(patched_seq)

    print(f'Seq {index}-{seq_id}: {raw_seq} (length: {seq_len})')
    print(f'Patched: \t  {patched_seq}')
    
    ref_input_ids = [CLS_TOKEN_ID] + [REF_TOKEN_ID] * seq_len + [SEP_TOKEN_ID]
    return raw_seq, tokenized_seq, tokenized_patched_seq, torch.tensor([ref_input_ids], device=device)

sample, tokenized_sample, tokenized_patched_sample, tokenized_baseline = get_sample(df, 3853)

Seq 3853-A0A2G6G956: MKIKIIAIGKLDSEVQKLFDEYQKRLGSFCKVETLFLKEDKNRQNNEQKVLEKSKSAFLIIFDEKGKEFTSQEFANFLEQKENQSVGEIVFFIGGTDGHTDFIKQNADFKLSLSKLTLPHKLACLFAVESIYRAFSIINHHPYHRE (length: 146)
Patched: 	  MKIKIIAIGKLDSEVQKLFDEYQKRLGSFCKVETLFLKEDKNRQNNEQKVLEKSKSAFLIIFDEKGKEFTSQEFANFLEQKENQSVGEIVFFIXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXAFSIINHHPYHRE


Get predictions for the tokenized sequence:

In [12]:
predict(tokenized_sample['input_ids'])

tensor([[-4.2254,  4.1789]], grad_fn=<AddmmBackward0>)

In [13]:
custom_forward(tokenized_sample['input_ids'])

tensor([0.0002], grad_fn=<UnsqueezeBackward0>)

There are 2 different ways of computing the attributions for emebedding layers. One option is to use `LayerIntegratedGradients` and compute the attributions with respect to `BertEmbedding`. The second option is to use `LayerIntegratedGradients` for each `word_embeddings`, `token_type_embeddings` and `position_embeddings` and compute the attributions w.r.t each embedding vector.

In [14]:
lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)

Helper function to summarize attributions for each word token in the sequence:

In [15]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions/ torch.norm(attributions)
    return attributions

In [18]:
def interpret_and_visualize(input_tensors, ref_input_ids, label, n_steps=1):
    input_ids = input_tensors['input_ids']
    attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True,
                                    n_steps=n_steps)
    # https://captum.ai/api/layer.html#layer-integrated-gradients
    # https://github.com/pytorch/captum/issues/538
    
    indices = input_ids[0].detach().tolist()
    
    all_tokens = tokenizer.convert_ids_to_tokens(indices)
    
    attributions_sum = summarize_attributions(attributions)
    attribution_scores = [round(_, 4) for _ in attributions_sum.tolist()]

    score = predict(input_ids)
    
    score_vis = viz.VisualizationDataRecord(
        word_attributions=attributions_sum, 
        pred_prob=float(tf.nn.softmax(score.cpu().detach().numpy())[0][int(score.argmax())]),
        pred_class=score.argmax().item(), 
        true_class=label, 
        attr_class=label,
        #attr_class=attribution_scores,  # Attribution Label
        attr_score=attributions_sum.sum(), 
        raw_input_ids=all_tokens, 
        convergence_score=delta)

    print('\033[1m', 'Visualization For Score', '\033[0m')
    viz.visualize_text([score_vis])
    
    del(input_ids)
    del(attributions)
    del(delta)
    del(ref_input_ids)
    del(indices)
    del(all_tokens)
    del(attributions_sum)
    del(attribution_scores)
    del(score)
    del(score_vis)

Observe the difference between different values used for `n_steps` (https://captum.ai/api/layer.html#captum.attr.LayerIntegratedGradients: "The number of steps used by the approximation method.").

*Default is 50.*

In [19]:
print('\nPositive class:\n')
interpret_and_visualize(tokenized_sample, tokenized_baseline, 1)       # very quick
interpret_and_visualize(tokenized_sample, tokenized_baseline, 1, 50)   # default
interpret_and_visualize(tokenized_sample, tokenized_baseline, 1, 128)  # too slow
print('\n\n\nPatched, negative class:\n')
interpret_and_visualize(tokenized_patched_sample, tokenized_baseline, 0)
interpret_and_visualize(tokenized_patched_sample, tokenized_baseline, 0, 50)
interpret_and_visualize(tokenized_patched_sample, tokenized_baseline, 0, 128)


Positive class:

 Visualization For Score 


 Visualization For Score 


 Visualization For Score 





Patched, negative class:

 Visualization For Score 


 Visualization For Score 


 Visualization For Score 


**How to interpret the colours:**

https://datascience.stackexchange.com/questions/87670/what-exactly-negative-positive-value-of-captums-integrated-gradient-mean

```
Positive attribution score means that the input in that particular position positively contributed to the final prediction and negative means the opposite. The magnitude of the attribution score signifies the strength of the contribution. Zero attribution score means no contribution from that particular feature.
```

*Github issue: https://github.com/pytorch/captum/issues/249#issuecomment-580569266*

*TL;DR: The output is the prediction probability (`p`) of being of the positive class. A negative class would be (`1 - p`). We attribute positive class probability (`p`) to the inputs of our model and in case something is predicted with high probability (as the positive class) we see many tokens that positively contribute to it.*

*When `p` is very low, there are no words contributing to the positive class. When we attribute to the positive class prob (`p`) we find words that pull away from it (influence it negatively). Those tokens are obviously the ones that pull towards the negative class with higher (`1-p`) probability.*

https://github.com/pytorch/captum/issues/249#issuecomment-580846266

*In a general case, red means that those tokens are pulling away from the positive class and most probably pulling towards the opposite class however I think that red might not always mean that it will always attribute to the other class. I think that's the assumption that we make here. We assume that the classifier is able to identify that a token is negatively correlated with the positive class so it must know something about that token, namely, that it is strongly pulling towards the opposite class (because there are no other options) and this is much easier to imagine for 2 class problem.*

## Other input sequences:

### Baseline

(Should have attribution score close to 0.)

In [20]:
baseline_tensor = torch.tensor([[2] + [0] * 15 + [3]], device=device)
interpret_and_visualize({'input_ids': baseline_tensor}, baseline_tensor, 0)

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (1.00),0,nan,[CLS] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [SEP]


## Visualize the attribution for more sequences:

In [19]:
import time

for i in range(5):
    sample, tokenized_sample, tokenized_patched_sample, tokenized_baseline = get_sample(df, i)
    print('\nPositive class:\n')
    interpret_and_visualize(tokenized_sample, tokenized_baseline, 1)       # very quick
    interpret_and_visualize(tokenized_sample, tokenized_baseline, 1, 50)   # default
    print('\n\n\nPatched, negative class:\n')
    interpret_and_visualize(tokenized_patched_sample, tokenized_baseline, 0)
    interpret_and_visualize(tokenized_patched_sample, tokenized_baseline, 0, 50)
    print('=' * 150 + '\n\n')
    
    del(sample)
    del(tokenized_sample)
    del(tokenized_patched_sample)
    del(tokenized_baseline)

Seq 0-A0A0X3YDV6: MVSKLGWQLVQLGRRLWVRTVAFAVLALFSALVAVLVQDYIPETLSDIIGAGAAENILNILATSMLTVTTFSLSVMVAAYSASSKDVSPRATRLLMEDSTTQNALATFVGSFLFSIVSIILLNTEVYNQRGRVVLFLATILVIVLIVVMILIWISHLSSLGRVGETAGKVEDQAYAALKRHNKYPWLQAQPLQSLQQIPKDAKPVLTNTIGYLQLIDIKAINKWAAENNCHAYVAVRPGVFVEPCRPLLWLSPAPADTSSVPLDAFCISNYRTFDQDPRFGLLVLAEIASKALSPAINDPGTAIEIIGRGMRVLSNWQAPSMDKATIQYPQVSVLPLSLTDLFDDFYTPIARDGAAMLEVQIRLQKSLLALAGQRAEFRPHTQRHSTAALARAEHAMAYSEDIKTLKALHQQLCDS (length: 416)
Patched: 	  MVSKLGWQLVQLGRRLWVRTVAFAVLALFSALVAVLVQDYIPETLSDIIGAGAAENILNILATSMLTVTTFSLSVMVAAYSASSKDVSPRATRLLMEDSTTQNALATFVGSFLFSIVSIILLNTEVYNQRGRVVLFLATILVIVLIVVMILIWISHLSSLGRVGETAGKVEDQAYAALKRHNKYPWLQAQPLQSLQQIPKDAKPVLTNTIGYLQLIDIKAINKWAAENNCHAYVAVRPGVFVEPCRPLLWLSPAPADTSSVPLDAFCISNYRTFDQDPRFGLLVLXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXTIQYPQVSVLPLSLTDLFDDFYTPIARDGAAMLEVQIRLQKSLLALAGQRAEFRPHTQRHSTAALARAEHAMAYSEDIKTLKALHQQLCDS

Positive class:

 Visualization For Score 


 Visualization For Score 





Patched, negative class:

 Visualization For Score 


 Visualization For Score 




Seq 1-J3D4J7: MANDFLFTSESVSEGHPDKVADQISDAILDAIFKQDPRSRVAAETLTNTGLVVLAGEITTNAHVDYIQVARDTIKRIGYDNTEYGIDYKGCAVMVCYDKQSNDIAQGVDHASDDHLNIGAGDQGLMFGYACDETPDLMPAPIYYAHRLVERQAQLRKDGRLPFLRPDAKSQVTMRYVDGKPHSIDTVVLSTQHSPDQSETPHKMKASFNEAIIEEIIKPVLPKGMLTKDTRYLINPTGRFVIGGPQGDCGLTGRKIIVDTYGGACPHGGGAFSGKDPSKVDRSAAYAARYVAKNIVAAGLARQ (length: 303)
Patched: 	  MANDFLFTSESVSEGHPDKVADQISDAILDAIFKQDPRSRVAAETLTNTGLVVLAGEITTNAHVDYIQVARDTIKRIGYDNTEYGIDYKGCAVMVCYDKQSNDIAQGVDHASDDHLNIGAGDQGLMFGYACDETPDLMPAPIYYAHRLVERQAQLRKDGRLPFLRPDAKSQVTMRYVDGKPHSIDTVVLSTQHSPDQSETPHKMKASFNEAIIEEIIKPVLPKGMLTKDTRYLINPTGRFVIGGPQGDCGLTGRKIIXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXAGLARQ

Positive class:

 Visualization For Score 


...

 Visualization For Score 




Seq 4-A0A3G9G4U1: MRRPVLLFTVAAFALMSAGLSSCGKPKAEHGDPHAEAAGEHGGGDHGGDGHGAAKGDVSHWSYEGDDGPAHWGDLSTENKACKTGPRQSPINLTGVAAPKSVNLTLDYTSSPAKIQNLGHAIQVSPTDGGGVVMDGVRYKLVQFHFHTPSEHTIDGHRAAIETHFVHKNDKGDLLVIGVLSDVGVADPMLAPIWTWLPTDPGPAALIPDLLVNARDLMPATEEFYAYSGSLTTPPCTEKVTWLVYASPLSISPEQVDAYQRLTGPNARPIQPPQGRDILHLVGS (length: 284)
Patched: 	  MRRPVLLFTVAAFALMSAGLSSCGKPKAEHGDPHAEAAGEHGGGDHGGDGHGAAKGDVSHWSYEGDDGPAHWGDLSTENKACKTGPRQSPINLTGVAAPKSVNLTLDYTSSPAKIQNLGHAIQVSPTDGGGVVMDGVRYKLVQFHFHTPSEHTIDGHRAAIETHFVHKNDKGDLLVIGVLSDVGVADPMLAPIWTWLPTDPGPAALIPDLLVNARDLMPATEEFYAYSGSLTTPPCTEKVXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXLVGS

Positive class:

 Visualization For Score 


 Visualization For Score 





Patched, negative class:

 Visualization For Score 


 Visualization For Score 


### MultiEmbedding Attribution

In [41]:
lig2 = LayerIntegratedGradients(custom_forward, \
                                [model.bert.embeddings.word_embeddings, \
                                 model.bert.embeddings.token_type_embeddings, \
                                 model.bert.embeddings.position_embeddings])
lig2

In [42]:
def construct_input_ref_token_type_pair(input_ids, sep_ind):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [54]:
_, tokenized_sample, _, tokenized_baseline = get_sample(df, 3853)
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(tokenized_sample['input_ids'], SEP_TOKEN_ID)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(tokenized_sample['input_ids'])
attention_mask = construct_attention_mask(tokenized_sample['input_ids'])
indices = tokenized_sample['input_ids'].detach().tolist()[0]
all_tokens = tokenizer.convert_ids_to_tokens(indices)

Seq 3853-A0A2G6G956: MKIKIIAIGKLDSEVQKLFDEYQKRLGSFCKVETLFLKEDKNRQNNEQKVLEKSKSAFLIIFDEKGKEFTSQEFANFLEQKENQSVGEIVFFIGGTDGHTDFIKQNADFKLSLSKLTLPHKLACLFAVESIYRAFSIINHHPYHRE (length: 146)
Patched: 	  MKIKIIAIGKLDSEVQKLFDEYQKRLGSFCKVETLFLKEDKNRQNNEQKVLEKSKSAFLIIFDEKGKEFTSQEFANFLEQKENQSVGEIVFFIXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXAFSIINHHPYHRE


In [44]:
attributions = lig2.attribute(inputs=(tokenized_sample['input_ids'], token_type_ids, position_ids),
                                    baselines=(tokenized_baseline, ref_token_type_ids, ref_position_ids),
                                    additional_forward_args=(attention_mask))
attributions

[tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [-1.3810e-12,  7.4986e-13, -3.1374e-12,  ...,  2.2875e-11,
            3.4599e-12,  8.1911e-12],
          [ 8.9521e-13,  1.0526e-12,  1.5316e-11,  ...,  3.9491e-11,
            1.6133e-12,  5.5295e-13],
          ...,
          [ 8.1135e-13,  5.5948e-13, -5.1659e-11,  ...,  5.2348e-11,
            2.8245e-12,  3.8956e-12],
          [ 1.8955e-12, -3.8470e-12, -4.9354e-11,  ...,  4.2340e-11,
           -9.6271e-12, -3.6969e-12],
          [-0.0000e+00,  0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
           -0.0000e+00, -0.0000e+00]]], dtype=torch.float64),
 tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [ 0.0000e+00, -0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [ 0.0000e+00, -0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           -0.0000e+00, -0.0000e+00]

In [45]:
attributions_word = summarize_attributions(attributions[0])
attributions_token_type = summarize_attributions(attributions[1])
attributions_position = summarize_attributions(attributions[2])

In [58]:
def get_topk_attributed_tokens(attrs, k=25):
    values, indices = torch.topk(attrs, k)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [59]:
top_words, top_words_val, top_word_ind = get_topk_attributed_tokens(attributions_word)
top_token_type, top_token_type_val, top_token_type_ind = get_topk_attributed_tokens(attributions_token_type)
top_pos, top_pos_val, pos_ind = get_topk_attributed_tokens(attributions_position)

df_topk = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(top_words, top_word_ind, top_words_val)],
                   'Token Type(Index), Attribution': ["{} ({}), {}".format(ttype, pos, round(val.item(),2)) for ttype, pos, val in zip(top_token_type, top_token_type_ind, top_words_val)],
                   'Position(Index), Attribution': ["{} ({}), {}".format(position, pos, round(val.item(),2)) for position, pos, val in zip(top_pos, pos_ind, top_pos_val)]})
df_topk

,"Word(Index), Attribution","Token Type(Index), Attribution","Position(Index), Attribution"
0,"C (30), 0.14","[SEP] (147), 0.14","[SEP] (147), 0.99"
1,"C (124), 0.14","K (2), 0.14","L (59), 0.0"
2,"E (146), 0.12","M (1), 0.12","F (58), 0.0"
3,"E (64), 0.12","[CLS] (0), 0.12","[CLS] (0), 0.0"
4,"E (47), 0.12","I (3), 0.12","A (57), -0.0"
5,"E (21), 0.12","E (73), 0.12","S (54), -0.0"
6,"E (68), 0.12","E (82), 0.12","E (82), -0.0"
7,"E (129), 0.12","D (101), 0.12","K (55), -0.0"
8,"E (33), 0.12","E (79), 0.12","I (60), -0.0"
9,"E (52), 0.12","D (97), 0.12","S (56), -0.0"
